In [1]:
import tensorflow as tf

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from PIL import Image

In [4]:
def create_class_weights(dataset,num_classes):
    class_weights = {}
    for i in range(dataset.shape[0]):
        for j in range(num_classes):
            if dataset.iloc[i][j]==1:
                if j in class_weights:
                    class_weights[j]+=1
                else:
                    class_weights[j]=1

    x=max(class_weights,key=class_weights.get)
    z=class_weights[x]
    
    for key in class_weights:
        class_weights[key]=int(z/class_weights[key])
    classes={}
    for j in range(num_classes):
        if dataset.columns.values[j] not in classes:
            classes[dataset.columns.values[j]] = j
    return classes,class_weights


In [5]:
gender_dataset = pd.read_csv('gender.csv').drop(columns = 'Unnamed: 0').set_index("JPG")
gender_dataset
gender_classes, gender_class_weights =create_class_weights(gender_dataset,2)

In [6]:
age_df = pd.read_csv("age.csv").drop(columns = 'Unnamed: 0').set_index("JPG")
age_df
age_classes, age_class_weights = create_class_weights(age_df,8)

In [7]:
ethnicity_df = pd.read_csv("ethnicity.csv").drop(columns = 'Unnamed: 0').set_index("JPG")
ethnicity_df
ethnicity_classes, ethnicity_class_weights = create_class_weights(ethnicity_df,5)


# Gender

# VGG16 w Regularization

In [9]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Gender_Images2/Train'
validation_folder = 'Gender_Images2/Val'
test_folder='Gender_Images2/Test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "binary", color_mode="rgb",classes = gender_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "binary", color_mode="rgb", classes = gender_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size=16, class_mode= "binary", color_mode="rgb",classes = gender_classes)

Found 14224 images belonging to 2 classes.
Found 4741 images belonging to 2 classes.
Found 4743 images belonging to 2 classes.


In [10]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [11]:
#Add regularization

import tempfile

def add_regularization(model, regularizer):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

reg = tf.keras.regularizers.l2(0.0001)
vgg16 = add_regularization(vgg16, reg)

In [12]:
#Check if regularization was added
vgg16.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0073838485>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.006740606>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.007655664>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008172592>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008904765>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0089925155>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00949263>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0119424565>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.013766433>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.015087408>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017912947>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.018236415>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017235568>]

In [13]:
for layer in vgg16.layers:
    layer.trainable = False
    
x = Flatten()(vgg16.output)
vgg_feedforward = Dense(4096,activation = 'relu')(x)
prediction = Dense(2, activation='softmax')(x)

model_gender = Model(inputs=vgg16.input, outputs=prediction)

In [14]:
model_gender.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
model_gender.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

gender_fit = model_gender.fit(
  train,
  validation_data=val,
  epochs=3,
  steps_per_epoch = 14224//16,
  validation_steps = 4741//16,
)

Epoch 1/3
889/889 [==============================] - 156s 167ms/step - loss: 0.6557 - accuracy: 0.8262 - val_loss: 0.4703 - val_accuracy: 0.8661
Epoch 2/3
889/889 [==============================] - 274s 308ms/step - loss: 0.4297 - accuracy: 0.8826 - val_loss: 0.4460 - val_accuracy: 0.8792accuracy: 
Epoch 3/3
889/889 [==============================] - 184s 206ms/step - loss: 0.3849 - accuracy: 0.8998 - val_loss: 0.4153 - val_accuracy: 0.8896


# VGG16 w/o Regularization

In [17]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Gender_Images2/Train'
validation_folder = 'Gender_Images2/Val'
test_folder='Gender_Images2/Test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "binary", color_mode="rgb",classes = gender_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "binary", color_mode="rgb", classes = gender_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size=16, class_mode= "binary", color_mode="rgb",classes = gender_classes)

Found 14224 images belonging to 2 classes.
Found 4741 images belonging to 2 classes.
Found 4743 images belonging to 2 classes.


In [18]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [19]:
for layer in vgg16.layers:
    layer.trainable = False
    
x = Flatten()(vgg16.output)
x = Dense(4096,activation = 'relu')(x)
prediction = Dense(2, activation='softmax')(x)

model_gender = Model(inputs=vgg16.input, outputs=prediction)

In [20]:
model_gender.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

gender_fit = model_gender.fit(
  train,
  validation_data=val,
  epochs=3,
  steps_per_epoch = 14224//16,
  validation_steps = 4741//16,
)

Epoch 1/3
889/889 [==============================] - 219s 245ms/step - loss: 0.4936 - accuracy: 0.8151 - val_loss: 0.3011 - val_accuracy: 0.8729
Epoch 2/3
889/889 [==============================] - 260s 293ms/step - loss: 0.2764 - accuracy: 0.8838 - val_loss: 0.3438 - val_accuracy: 0.8666
Epoch 3/3
889/889 [==============================] - 295s 332ms/step - loss: 0.2251 - accuracy: 0.9062 - val_loss: 0.2918 - val_accuracy: 0.8902


# Resnet50 w Regularization

In [11]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Gender_Images2/Train'
validation_folder = 'Gender_Images2/Val'
test_folder='Gender_Images2/Test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "binary", color_mode="rgb",classes = gender_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "binary", color_mode="rgb", classes = gender_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size=16, class_mode= "binary", color_mode="rgb",classes = gender_classes)

Found 14224 images belonging to 2 classes.
Found 4741 images belonging to 2 classes.
Found 4743 images belonging to 2 classes.


In [12]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

ResNet50 = ResNet50V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [13]:
#Add regularization

import tempfile

def add_regularization(model, regularizer):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

reg = tf.keras.regularizers.l2(0.0001)
ResNet50 = add_regularization(ResNet50, reg)

C:\Users\bohne\anaconda3\envs\my_env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [14]:
#Check if regularization was added!
ResNet50.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.009224404>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0011331815>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.001809771>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0052976073>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0015453106>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0014674503>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0021098019>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0019758209>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0009776982>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0017850538>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0013890573>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0015911335>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0048106033>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0064114067>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00419335>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0020050437>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0046226853>,
 

In [16]:
#Freeze Layers
for layer in ResNet50.layers:
    layer.trainable = False

x = Flatten()(ResNet50.output)
x = Dense(1024,activation = 'relu')(x)
prediction = Dense(2, activation='softmax')(x)
model_ResNet_gender = Model(inputs= ResNet50.input, outputs=prediction)
model_ResNet_gender.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [21]:
model_ResNet_gender.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

model_ResNet_fit = model_ResNet_gender.fit(
    train,
    validation_data=val, 
    epochs=3, 
    steps_per_epoch=14224//16,
  validation_steps = 4741//16
)

Epoch 1/3
889/889 [==============================] - 182s 199ms/step - loss: 0.7307 - accuracy: 0.9229 - val_loss: 0.8378 - val_accuracy: 0.8875
Epoch 2/3
889/889 [==============================] - 410s 461ms/step - loss: 0.6214 - accuracy: 0.9609 - val_loss: 0.9111 - val_accuracy: 0.8813
Epoch 3/3
889/889 [==============================] - 333s 375ms/step - loss: 0.5732 - accuracy: 0.9769 - val_loss: 1.0370 - val_accuracy: 0.8866


# ResNet w/o Regularization

In [8]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Gender_Images2/Train'
validation_folder = 'Gender_Images2/Val'
test_folder='Gender_Images2/Test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "binary", color_mode="rgb",classes = gender_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "binary", color_mode="rgb", classes = gender_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size=16, class_mode= "binary", color_mode="rgb",classes = gender_classes)

Found 14224 images belonging to 2 classes.
Found 4741 images belonging to 2 classes.
Found 4743 images belonging to 2 classes.


In [9]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

ResNet50 = ResNet50V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [10]:
#Freeze Layers
for layer in ResNet50.layers:
    layer.trainable = False

x = Flatten()(ResNet50.output)
x = Dense(1024,activation = 'relu')(x)
prediction = Dense(2, activation='softmax')(x)
model_ResNet_gender = Model(inputs= ResNet50.input, outputs=prediction)
model_ResNet_gender.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [11]:
model_ResNet_gender.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

model_ResNet_fit = model_ResNet_gender.fit(
    train,
    validation_data=val, 
    epochs=3, 
    steps_per_epoch=14224//16,
  validation_steps = 4741//16
)

Epoch 1/3
889/889 [==============================] - 77s 76ms/step - loss: 1.1653 - accuracy: 0.8429 - val_loss: 0.2807 - val_accuracy: 0.8881
Epoch 2/3
889/889 [==============================] - 117s 131ms/step - loss: 0.2087 - accuracy: 0.9182 - val_loss: 0.3191 - val_accuracy: 0.8830
Epoch 3/3
889/889 [==============================] - 150s 169ms/step - loss: 0.1407 - accuracy: 0.9455 - val_loss: 0.3243 - val_accuracy: 0.8847


# Age

# VGG16 AGE w Regularization

In [12]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Age_Images2/Train'
validation_folder = 'Age_Images2/Val'
test_folder='Age_Images2/Test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb",classes = age_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb", classes = age_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb",classes = age_classes)

Found 14223 images belonging to 8 classes.
Found 4742 images belonging to 8 classes.
Found 4743 images belonging to 8 classes.


In [13]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [14]:
#Add regularization

import tempfile

def add_regularization(model, regularizer):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

reg = tf.keras.regularizers.l2(0.0001)
vgg16 = add_regularization(vgg16, reg)

In [15]:
#Check of regularization was added!
vgg16.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0073838485>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.006740606>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.007655664>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008172592>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008904765>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0089925155>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00949263>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0119424565>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.013766433>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.015087408>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017912947>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.018236415>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017235568>]

In [16]:
for layer in vgg16.layers:
    layer.trainable = False
    
x = Flatten()(vgg16.output)
x = Dense(4096,activation = 'relu')(x)
prediction = Dense(8, activation='softmax')(x)

model_age = Model(inputs=vgg16.input, outputs=prediction)

In [17]:
model_age.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [18]:
model_age.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss')

vgg16_fit = model_age.fit(
  train,
  validation_data=val,
  epochs=4,
  steps_per_epoch = 14224//8,
  validation_steps = 4741//8
  
)#callbacks=[es]

Epoch 1/4
1778/1778 [==============================] - 194s 108ms/step - loss: 1.6228 - accuracy: 0.5135 - val_loss: 1.2849 - val_accuracy: 0.5602
Epoch 2/4
1778/1778 [==============================] - 166s 93ms/step - loss: 1.1764 - accuracy: 0.5862 - val_loss: 1.1919 - val_accuracy: 0.5872
Epoch 3/4
1778/1778 [==============================] - 195s 109ms/step - loss: 1.0715 - accuracy: 0.6226 - val_loss: 1.2317 - val_accuracy: 0.5963
Epoch 4/4
1778/1778 [==============================] - 193s 108ms/step - loss: 1.0013 - accuracy: 0.6495 - val_loss: 1.1973 - val_accuracy: 0.5929


# Vgg16 w/o Regularization

In [8]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Age_Images2/Train'
validation_folder = 'Age_Images2/Val'
test_folder='Age_Images2/Test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb",classes = age_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb", classes = age_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb",classes = age_classes)

Found 14223 images belonging to 8 classes.
Found 4742 images belonging to 8 classes.
Found 4743 images belonging to 8 classes.


In [9]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
vgg16_conv = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [10]:
for layer in vgg16_conv.layers:
    layer.trainable = False
    
vgg_feedforward = Flatten()(vgg16_conv.output)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)

prediction = Dense(8, activation='softmax')(vgg_feedforward)


model_age = Model(inputs=vgg16_conv.input, outputs=prediction)

In [11]:
model_age.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss')

vgg16_fit = model_age.fit(
  train,
  validation_data=val,
  epochs=4,
  steps_per_epoch = 14224//8,
  validation_steps = 4741//8
  
)#callbacks=[es]

Epoch 1/4
1778/1778 [==============================] - 169s 92ms/step - loss: 1.3659 - accuracy: 0.5113 - val_loss: 1.1423 - val_accuracy: 0.5304
Epoch 2/4
1778/1778 [==============================] - 256s 144ms/step - loss: 1.0538 - accuracy: 0.5767 - val_loss: 1.0191 - val_accuracy: 0.5935
Epoch 3/4
1778/1778 [==============================] - 292s 164ms/step - loss: 0.9692 - accuracy: 0.6050 - val_loss: 1.0329 - val_accuracy: 0.5906
Epoch 4/4
1778/1778 [==============================] - 239s 135ms/step - loss: 0.9004 - accuracy: 0.6272 - val_loss: 1.0377 - val_accuracy: 0.5817


# ResNet50 w Regularization

In [12]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Age_Images2/Train'
validation_folder = 'Age_Images2/Val'
test_folder='Age_Images2/Test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb",classes = age_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb", classes = age_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb",classes = age_classes)

Found 14223 images belonging to 8 classes.
Found 4742 images belonging to 8 classes.
Found 4743 images belonging to 8 classes.


In [13]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

ResNet50 = ResNet50V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [14]:
#Add regularization

import tempfile

def add_regularization(model, regularizer):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

reg = tf.keras.regularizers.l2(0.0001)
ResNet50 = add_regularization(ResNet50, reg)

C:\Users\bohne\anaconda3\envs\my_env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [15]:
ResNet50.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.009224404>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0011331815>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.001809771>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0052976073>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0015453106>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0014674503>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0021098019>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0019758209>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0009776982>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0017850538>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0013890573>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0015911335>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0048106033>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0064114067>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00419335>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0020050437>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0046226853>,
 

In [16]:
for layer in ResNet50.layers:
    layer.trainable = False
    
x = Flatten()(ResNet50.output)
x = Dense(1024,activation = 'relu')(x)
prediction = Dense(8, activation='softmax')(x)
model_ResNet_age = Model(inputs= ResNet50.input, outputs=prediction)
model_ResNet_age.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [17]:

model_ResNet_age.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

model_ResNet_fit = model_ResNet_age.fit(
    train,
    validation_data=val, 
    epochs=5, 
    steps_per_epoch=14224//8,
  validation_steps = 4741//8
)

Epoch 1/5
1778/1778 [==============================] - 184s 98ms/step - loss: 2.5633 - accuracy: 0.5162 - val_loss: 1.8512 - val_accuracy: 0.5462
Epoch 2/5
1778/1778 [==============================] - 332s 187ms/step - loss: 1.4874 - accuracy: 0.6013 - val_loss: 1.6681 - val_accuracy: 0.5697
Epoch 3/5
1778/1778 [==============================] - 532s 299ms/step - loss: 1.3102 - accuracy: 0.6709 - val_loss: 1.9010 - val_accuracy: 0.5441
Epoch 4/5
1778/1778 [==============================] - 341s 192ms/step - loss: 1.1550 - accuracy: 0.7381 - val_loss: 1.8860 - val_accuracy: 0.5629
Epoch 5/5
1778/1778 [==============================] - 307s 173ms/step - loss: 1.0630 - accuracy: 0.7913 - val_loss: 2.4562 - val_accuracy: 0.5648


# ResNet50 w/o Regularization

In [18]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Age_Images2/Train'
validation_folder = 'Age_Images2/Val'
test_folder='Age_Images2/Test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb",classes = age_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb", classes = age_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb",classes = age_classes)

Found 14223 images belonging to 8 classes.
Found 4742 images belonging to 8 classes.
Found 4743 images belonging to 8 classes.


In [19]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

ResNet50 = ResNet50V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in ResNet50.layers:
    layer.trainable = False
    
x = Flatten()(ResNet50.output)
x = Dense(1024,activation = 'relu')(x)
prediction = Dense(8, activation='softmax')(x)
model_ResNet_age = Model(inputs= ResNet50.input, outputs=prediction)
model_ResNet_age.summary()

In [ ]:

model_ResNet_age.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

model_ResNet_fit = model_ResNet_age.fit(
    train,
    validation_data=val, 
    epochs=3, 
    steps_per_epoch=14224//8,
  validation_steps = 4741//8
)

Epoch 1/5
1778/1778 [==============================] - 189s 95ms/step - loss: 1.8030 - accuracy: 0.5230 - val_loss: 1.1240 - val_accuracy: 0.5608
Epoch 2/5
1778/1778 [==============================] - 143s 80ms/step - loss: 0.9636 - accuracy: 0.6052 - val_loss: 1.2926 - val_accuracy: 0.5481
Epoch 3/5
1778/1778 [==============================] - 226s 127ms/step - loss: 0.7811 - accuracy: 0.6770 - val_loss: 1.2974 - val_accuracy: 0.5621
Epoch 4/5
1778/1778 [==============================] - 168s 95ms/step - loss: 0.6449 - accuracy: 0.7429 - val_loss: 1.4546 - val_accuracy: 0.5766s
Epoch 5/5
 893/1778 [==============>...............] - ETA: 58s - loss: 0.4890 - accuracy: 0.8103

## Ethnicity

# VGG16 w/o Regularization

In [15]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Ethnicity_Images/train'
validation_folder = 'Ethnicity_Images/val'
test_folder='Ethnicity_Images/test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "sparse", color_mode="rgb",classes = ethnicity_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size = 16, class_mode= "sparse", color_mode="rgb", classes = ethnicity_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size = 16, class_mode= "sparse", color_mode="rgb",classes = ethnicity_classes)

Found 13206 images belonging to 5 classes.
Found 4403 images belonging to 5 classes.
Found 4404 images belonging to 5 classes.


In [16]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)



In [17]:
for layer in vgg16_conv.layers:
    layer.trainable = False
    
x = Flatten()(vgg16.output)
x = Dense(4096,activation = 'relu')(x)
prediction = Dense(5, activation='softmax')(x)

model_ethnicity = Model(inputs=vgg16.input, outputs=prediction)

In [18]:
model_ethnicity.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [19]:
model_ethnicity.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

vgg16_fit = model_ethnicity.fit(
  train,
  validation_data=val,
  epochs=4,
  steps_per_epoch = 13206//16,
  validation_steps = 4403//16,
  callbacks=[es]
)

Epoch 1/4
825/825 [==============================] - 149s 176ms/step - loss: 1.0090 - accuracy: 0.6657 - val_loss: 0.6676 - val_accuracy: 0.7448
Epoch 2/4
825/825 [==============================] - 108s 131ms/step - loss: 0.5761 - accuracy: 0.7826 - val_loss: 0.6580 - val_accuracy: 0.7500
Epoch 3/4
825/825 [==============================] - 121s 147ms/step - loss: 0.4663 - accuracy: 0.8267 - val_loss: 0.6245 - val_accuracy: 0.7802
Epoch 4/4
825/825 [==============================] - 147s 178ms/step - loss: 0.3979 - accuracy: 0.8510 - val_loss: 0.8040 - val_accuracy: 0.7436


# VGG16 w Regularization

In [15]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Ethnicity_Images/train'
validation_folder = 'Ethnicity_Images/val'
test_folder='Ethnicity_Images/test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "sparse", color_mode="rgb",classes = ethnicity_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size = 16, class_mode= "sparse", color_mode="rgb", classes = ethnicity_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size = 16, class_mode= "sparse", color_mode="rgb",classes = ethnicity_classes)

Found 13206 images belonging to 5 classes.
Found 4403 images belonging to 5 classes.
Found 4404 images belonging to 5 classes.


In [16]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)



In [17]:
#Add regularization

import tempfile

def add_regularization(model, regularizer):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

reg = tf.keras.regularizers.l2(0.0001)
vgg16 = add_regularization(vgg16, reg)

In [18]:
vgg16.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0073838485>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.006740606>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.007655664>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008172592>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008904765>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0089925155>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00949263>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0119424565>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.013766433>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.015087408>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017912947>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.018236415>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017235568>]

In [19]:
for layer in vgg16.layers:
    layer.trainable = False
    
x = Flatten()(vgg16.output)
x = Dense(4096,activation = 'relu')(x)
prediction = Dense(5, activation='softmax')(x)

model_ethnicity = Model(inputs=vgg16.input, outputs=prediction)
model_ethnicity.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [20]:
model_ethnicity.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

vgg16_fit = model_ethnicity.fit(
  train,
  validation_data=val,
  epochs=4,
  steps_per_epoch = 13206//16,
  validation_steps = 4403//16,
  callbacks=[es]
)

Epoch 1/4
825/825 [==============================] - 74s 87ms/step - loss: 1.4187 - accuracy: 0.6648 - val_loss: 0.7920 - val_accuracy: 0.7450


# ResNet50 w/o Regularization

In [8]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Ethnicity_Images/train'
validation_folder = 'Ethnicity_Images/val'
test_folder='Ethnicity_Images/test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "sparse", color_mode="rgb",classes = ethnicity_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size = 16, class_mode= "sparse", color_mode="rgb", classes = ethnicity_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size = 16, class_mode= "sparse", color_mode="rgb",classes = ethnicity_classes)

Found 13206 images belonging to 5 classes.
Found 4403 images belonging to 5 classes.
Found 4404 images belonging to 5 classes.


In [9]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
ResNet50 = ResNet50V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [10]:

for layer in ResNet50.layers:
    layer.trainable = False
    
x = Flatten()(ResNet50.output)
x = Dense(1024,activation = 'relu')(x)
prediction = Dense(8, activation='softmax')(x)
model_ResNet50 = Model(inputs= ResNet50.input, outputs=prediction)
model_ResNet50.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [11]:

model_ResNet50.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

model_ResNet_fit = model_ResNet50.fit(
    train,
    validation_data=val, 
    epochs=2, 
    steps_per_epoch=13206//16,
    validation_steps=4403//16)

Epoch 1/2
825/825 [==============================] - 72s 74ms/step - loss: 2.5278 - accuracy: 0.7103 - val_loss: 0.6171 - val_accuracy: 0.7720
Epoch 2/2
825/825 [==============================] - 113s 137ms/step - loss: 0.3875 - accuracy: 0.8601 - val_loss: 0.7319 - val_accuracy: 0.7611


# ResNet50 w Regularization

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Ethnicity_Images/train'
validation_folder = 'Ethnicity_Images/val'
test_folder='Ethnicity_Images/test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "sparse", color_mode="rgb",classes = ethnicity_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size = 16, class_mode= "sparse", color_mode="rgb", classes = ethnicity_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size = 16, class_mode= "sparse", color_mode="rgb",classes = ethnicity_classes)

In [15]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
ResNet50 = ResNet50V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [16]:
#Add regularization

import tempfile

def add_regularization(model, regularizer):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

reg = tf.keras.regularizers.l2(0.001)
ResNet50 = add_regularization(ResNet50, reg)

In [17]:
ResNet50.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.092244044>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.011331816>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.018097712>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.052976076>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.015453108>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.014674504>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.02109802>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.01975821>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.009776983>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017850539>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.013890573>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.015911337>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.048106037>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.06411407>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.041933503>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.020050438>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.046226855>,
 <tf.Tensor: shap

In [18]:

for layer in ResNet50.layers:
    layer.trainable = False
    
x = Flatten()(ResNet50.output)
x = Dense(1024,activation = 'relu')(x)
prediction = Dense(8, activation='softmax')(x)
model_ResNet50 = Model(inputs= ResNet50.input, outputs=prediction)
model_ResNet50.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [20]:

model_ResNet50.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

model_ResNet_fit = model_ResNet50.fit(
    train,
    validation_data=val, 
    epochs=2, 
    steps_per_epoch=13206//16,
    validation_steps=4403//16)

Epoch 1/2
825/825 [==============================] - 194s 224ms/step - loss: 5.7275 - accuracy: 0.7956 - val_loss: 5.7403 - val_accuracy: 0.7816
Epoch 2/2
825/825 [==============================] - 244s 296ms/step - loss: 5.3694 - accuracy: 0.9052 - val_loss: 5.8646 - val_accuracy: 0.7739
